# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

### Содержание:
* [Откроем и изучим файл](#1-section)
* [Разобьем данные на выборки.](#2-section)
* [Исследуем модели.](#3-section)
* [Проверим модель на тестовой выборке.](#4-section)
* [Проверим модель на адекватность.](#5-section)
* [Выводю](#6-section)

## Откроем и изучим файл<a id='1-section'></a>

In [5]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [6]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [7]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [9]:
df['messages'] = df['messages'].astype('int')

In [10]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [11]:
df.duplicated().sum()

0

Данные находятся в идеальном состоянии, отстутсвуют пропуски, нет дубликатов.

## Разобьем данные на выборки <a id='2-section'></a>

In [12]:
df_train, df_test = train_test_split(df, test_size=0.4, random_state=12345)
features = df_test.drop('is_ultra', axis=1)
target = df_test['is_ultra']

In [13]:
features_test, features_valid, target_test, target_valid = train_test_split(
    features, target, test_size=.25, random_state=12345)

<div class="alert alert-success">
<b>ОТЛИЧНО! 👍</b>

1. Здесь хорошо, происходит фиксация random_state. Воспроизводимость результатов разбиения выборки на обучающую (тренировочную) / тестовую / валидационную мы обеспечили, значит при всех последующих запусках нашего кода подвыборки будут идентичными.
    
2. Доли размеров train/test/valid 3:1:1 - хорошо.
</div>

In [14]:
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']

In [37]:
print(features_train.shape)
print(target_train.shape)

(1928, 4)
(1928,)


## Исследуем модели<a id='3-section'></a>

**Логистическая регрессия**

In [16]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
print("Качество модели:", accuracy)

Качество модели: 0.6956521739130435


**Дерево решений**

In [17]:
best_model = 0
best_accuracy = 0
best_depth = 0
for depth in range(1,6):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    accuracy = model.score(features_valid, target_valid)
    if accuracy > best_model:
        best_accuracy = accuracy
        best_depth = depth
print("Глубина дерева:", best_depth, "Качество модели:", best_accuracy)

Глубина дерева: 5 Качество модели: 0.8012422360248447


**Случайный лес**

In [18]:
best_accuracy = 0
best_depth = 0
for depth in range(1,11):
    model = RandomForestClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    accuracy = model.score(features_valid, target_valid)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth
print("Глубина дерева:", best_depth, "Качество модели:", best_accuracy)

Глубина дерева: 10 Качество модели: 0.8167701863354038


## Проверим модель на тестовой выборке <a id='4-section'></a>

**Логистическая регрессия**

In [19]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество модели на тестовой выборке:", accuracy)

Качество модели на тестовой выборке: 0.6981327800829875


**Дерево решений**

In [20]:
model = DecisionTreeClassifier(max_depth=5, random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.774896265560166


**Случайный лес**

In [21]:
model = RandomForestClassifier(max_depth=8, random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7977178423236515


## Проверим модели на адекватность <a id='5-section'></a>

In [22]:
dummy_clf = DummyClassifier()
dummy_clf.fit(features_train, target_train)
dm = dummy_clf.score(features_test, target_test)
print("Качество наивного прогноза :", dm)

Качество наивного прогноза : 0.6950207468879668


In [18]:
dummy_clf = DummyClassifier(strategy="stratified")
dummy_clf.fit(features_train, target_train)
dm = dummy_clf.score(features_test, target_test)
print("Качество наивного прогноза :", dm)

Качество наивного прогноза : 0.5695020746887967


In [19]:
dummy_clf = DummyClassifier(strategy="uniform")
dummy_clf.fit(features_train, target_train)
dm = dummy_clf.score(features_test, target_test)
print("Качество наивного прогноза :", dm)

Качество наивного прогноза : 0.5


# Вывод <a id='6-section'></a>

В ходе проделанного исследования, были проверены три модели. Наилучший результат показала модель "Случайный лес" (0.81).